In [125]:
import numpy as np
import pandas as pd
import seaborn as sn
import os
import sklearn as sk
import statsmodels.api as sm

code_map = {'BEN':'BENIN','BFA':'BURKINA FASO','CIV':'COTE D\'IVOIRE','GNB':'GUINEE BISSAU','MLI':'MALI','NER':'NIGER','SEN':'SENEGAL','TGO':'TOGO'}


class WAEMU:
    def __init__(self):
        self.data = pd.read_stata('https://github.com/mermozhk/Poverty/raw/main/WAEMU_data/WAEMU_data.dta')
        self.data['zref_intl'] = 2.15*365
        self.data['weight'] = self.data['hhweight']*self.data['hhsize']
        self.dummies = pd.get_dummies(self.data.CY)
        self.data = pd.concat([self.data, self.dummies], axis='columns')
    
    def estimation_procedure(self,index):
        wls_model = sm.WLS(self.data[index],self.dummies, weights=self.data['weight'])
        return wls_model.fit()
    
    def prevalence(self):
        self.data['prevalence_ind'] =(self.data['dtot_ppa_ind']<self.data['zref_intl']).map({False:0, True:1})
        return wls_object(self.estimation_procedure('prevalence_ind'))
    
      
    def gap(self):
        if not('prevalence_ind' in self.data.columns):
            
            self.data['prevalence_ind'] =(self.data['dtot_ppa_ind']<self.data['zref_intl']).map({False:0, True:1})
         
        self.data['gap_ind'] = (1-self.data['dtot_ppa_ind']/(self.data['zref_intl']))*self.data['prevalence_ind']
        return wls_object(self.estimation_procedure('gap_ind'))
    
    def severity(self):
        if not('prevalence_ind' in self.data.columns):
            
            self.data['prevalence_ind'] =(self.data['dtot_ppa_ind']<self.data['zref_intl']).map({False:0, True:1})
            
        self.data['severity_ind'] = ((1-self.data['dtot_ppa_ind']/(self.data['zref_intl']))**2)*self.data['prevalence_ind']
        
        return wls_object(self.estimation_procedure('severity_ind'))
        
        
class wls_object:
    def __init__(self,fitting_output):
        self.object = fitting_output
        
    def print_result(self):
        return print(self.object.summary())
        
    def to_table(self,index,columns,by_table=False):
        #self.object.fittedvalues.unique()
        data=pd.DataFrame({'Country':columns,'Index':self.object.fittedvalues.unique()})
        data.columns = ['Country',index]
        data['year'] = data['Country'].apply(lambda x:x[-4:])
        data['Code']=data['Country'].apply(lambda x:x[:3])
        data['Country Name']=data['Code'].map(code_map)
        df=data.drop(columns=['Country','Code'])
        if not by_table:
            return df
        else:
            return df.pivot(index='year',columns='Country Name')
        

In [126]:
wa = WAEMU() # Crée un objet WAEMU comportant la base de données

In [127]:
# Exemple de calcul de prévalence
resultat_prevalence = wa.prevalence()

In [128]:
#Affichage simple des résultats
resultat_prevalence.print_result()

                            WLS Regression Results                            
Dep. Variable:         prevalence_ind   R-squared:                       0.131
Model:                            WLS   Adj. R-squared:                  0.131
Method:                 Least Squares   F-statistic:                     1203.
Date:                Mon, 01 Jul 2024   Prob (F-statistic):               0.00
Time:                        08:39:30   Log-Likelihood:                -88250.
No. Observations:              119189   AIC:                         1.765e+05
Df Residuals:                  119173   BIC:                         1.767e+05
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
BEN2018        0.1874      0.005     36.234      0.0

In [129]:
#
resultat_prevalence.to_table('Prevalence',wa.dummies.columns)
#wa.dummies.columns
#res.object.fittedvalues.unique()

,Prevalence,year,Country Name
0,0.187436,2018,BENIN
1,0.085756,2021,BENIN
2,0.330190,2018,BURKINA FASO
3,0.184733,2021,BURKINA FASO
4,0.122367,2018,COTE D'IVOIRE
5,0.066431,2021,COTE D'IVOIRE
6,0.205967,2018,GUINEE BISSAU
7,0.168300,2021,GUINEE BISSAU
8,0.167538,2018,MALI
9,0.156681,2021,MALI


In [130]:
resultat_severite = wa.severity()
resultat_severite.to_table('Severity',wa.dummies.columns)

,Severity,year,Country Name
0,0.015143,2018,BENIN
1,0.006195,2021,BENIN
2,0.034855,2018,BURKINA FASO
3,0.013980,2021,BURKINA FASO
4,0.008477,2018,COTE D'IVOIRE
5,0.003507,2021,COTE D'IVOIRE
6,0.014033,2018,GUINEE BISSAU
7,0.009518,2021,GUINEE BISSAU
8,0.011078,2018,MALI
9,0.009002,2021,MALI


In [131]:
resultat_gap = wa.gap()
resultat_gap.to_table('Gap',wa.dummies.columns)

,Gap,year,Country Name
0,0.043599,2018,BENIN
1,0.018132,2021,BENIN
2,0.090497,2018,BURKINA FASO
3,0.041804,2021,BURKINA FASO
4,0.025837,2018,COTE D'IVOIRE
5,0.012056,2021,COTE D'IVOIRE
6,0.043921,2018,GUINEE BISSAU
7,0.032666,2021,GUINEE BISSAU
8,0.034772,2018,MALI
9,0.030806,2021,MALI
